In [1]:
import numpy as np
import tensorflow as tf
from iisignature_tensorflow_2 import SigLayer, LogSigLayer, SigScaleLayer, SigJoinLayer

2024-05-29 17:51:39.660745: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 17:51:39.667161: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 17:51:39.735646: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 17:51:41.868491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print('generating inputs and targets')
#random inputs
# Generate time series data
def generate_time_series(batch_size, n_steps, n_features):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))  # wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)  # noise
    series = series[..., np.newaxis]  # Add feature dimension
    series = np.repeat(series, n_features, axis=-1)  # Repeat features
    return series

# Random inputs
x = generate_time_series(10000, 20, 7)

# Generate y by passing through a network 
generate_y_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=x.shape[1:]), 
    tf.keras.layers.Dense(100, activation='sigmoid'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.GRU(100, activation='tanh', return_sequences=True),
    tf.keras.layers.GRU(100),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='linear')
])

y = generate_y_model(x)

print('Sequential Example')

# Standard keras example
default_sequential_example_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=x.shape[1:]), 
    tf.keras.layers.Dense(10, activation='linear'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(y.shape[-1], activation='linear')
], name = 'default_sequential')
default_sequential_example_model.summary()

default_sequential_example_model.compile('adam', 'mse')
default_sequential_example_model.fit(x, y, epochs=20, validation_split=.2)


print('SigLayer Example')
# Using SigLayer example
sig_example_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=x.shape[1:]), 
    tf.keras.layers.Dense(10, activation='linear'),
    SigLayer(2),
    tf.keras.layers.Dense(y.shape[-1], activation='linear')
], name = "sig_example")
sig_example_model.summary()

sig_example_model.compile('adam', 'mse')
sig_example_model.fit(x, y, epochs=20, validation_split=.2)

print('LogSigLayer Example')
# Using LogSig Layer example
logsig_example_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=x.shape[1:]), 
    tf.keras.layers.Dense(10, activation='linear'),
    LogSigLayer(2),
    tf.keras.layers.Dense(y.shape[-1], activation='linear')
], name = 'logsig_example')
logsig_example_model.summary()

logsig_example_model.compile('adam', 'mse')
logsig_example_model.fit(x, y, epochs=20, validation_split=.2)


print('SigScale Example')
# Using SigScale Layer example
def create_sigscale_example(x, y):
    inputs = tf.keras.Input(shape=x.shape[1:])
    embeded_inputs = tf.keras.layers.Dense(10, activation="linear")(inputs)
    sig_1 = SigLayer(2)(embeded_inputs)
    scale = tf.keras.layers.Lambda(lambda x: x[:,-1,:])(embeded_inputs) #Probably a bad chocie, just made to have something the good shape
    sigscale = SigScaleLayer(2)([sig_1, scale])
    output = tf.keras.layers.Dense(y.shape[-1], activation='linear')(sigscale)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

sigscale_example_model = create_sigscale_example(x, y)
sigscale_example_model.summary()

sigscale_example_model.compile(optimizer='adam', loss='mse')
sigscale_example_model.fit(x, y, epochs=20, validation_split=.2)       

print('SigJoin Example')
# Using SigScale Layer example
def create_sigjoin_example(x, y):
    inputs = tf.keras.Input(shape=x.shape[1:])
    embeded_inputs = tf.keras.layers.Dense(10, activation="linear")(inputs)
    sig_1 = SigLayer(2)(embeded_inputs)
    scale = tf.keras.layers.Lambda(lambda x: x[:,-1,:])(embeded_inputs) #Probably a bad chocie, just made to have something the good shape
    sigscale = SigJoinLayer(2)([sig_1, scale])
    output = tf.keras.layers.Dense(y.shape[-1], activation='linear')(sigscale)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

sigscale_example_model = create_sigjoin_example(x, y)
sigscale_example_model.summary()

sigscale_example_model.compile(optimizer='adam', loss='mse')
sigscale_example_model.fit(x, y, epochs=20, validation_split=.2)      

generating inputs and targets


2024-05-29 17:51:43.301687: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Sequential Example


Model: "default_sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 20, 10)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           603 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 20, 7)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 20, 10)    │         80 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sig_layer_2         │ (None, 110)       │          0 │ dense_11[0][0]    │
│ (SigLayer)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 10)        │          0 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sig_join_layer      │ (None, 110)       │          0 │ sig_layer_2[0][0… │
│ (SigJoinLayer)      │                   │            │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 3)         │        333 │ sig_join_layer[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 413 (1.61 KB)

 Trainable params: 413 (1.61 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0268 - val_loss: 3.3934e-05
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2.2396e-05 - val_loss: 9.9960e-06
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 9.7232e-06 - val_loss: 9.6828e-06
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.7315e-06 - val_loss: 8.9866e-06
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 9.7077e-06 - val_loss: 9.1955e-06
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.3482e-05 - val_loss: 1.1424e-05
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.8852e-06 - val_loss: 1.0256e-05
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.1165e-05 - val_loss: 1.1119e-05
Epoch 9/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.0395e-05 - val_loss: 9.2422e-06
Epoch 10/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.0750e-05 - val_loss: 1.1467e-05
Epoch 11/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 1.